In [2]:
def check_if_smooth(n, prod_factor_base):
    """
    checks to see if n can be factored over a given factor base
    running time is y^1+o(1) where y is the y-smooth
    """

    g = gcd(n, prod_factor_base)

    if g > 1:                    # then n = rg^e for some e >= 1

        # want to find the first e such that g^e | n
        e = 1
        r = n / g^e

        # increment e until r lives in Z
        while r not in ZZ:
            e += 1
            r = n / g^e

        if r == 1:               # then n is smooth
            return True

        elif r != 1:             # recurse
            return check_if_smooth(r, prod_factor_base)

    else:
        return False


def index_calc(g, q, h, bound):
    """
    input discrete log generator g, modulus q, argument h, factor base bound {-1,2,3,5,7,11,...,p_r} : p_r < bound
    output x such that g^x = h (mod q)
    """

    # get the factor base as a list
    factor_base = list(primes(bound))

    # compute the product of the factor base, this is used in the smoothness checking algorithm
    prd = prod(factor_base)

    relations = []               # 2d list to be populated with relation vectors


    g_k = 1
    for k in range(1, q-1):

        # compute g^k
        #g_k = g^k % q
        g_k = g_k*g % q

        # check if g_k can be factored over the factor base, if it cannot, skip until it can
        if check_if_smooth(g_k, prd):

            # we want to find e_i's such that g^k mod q = (-1)^e_0 * 2^e_1 * 3, ... , p_r^e_r

            g_k_factorisation = list(factor(g_k))    # first get the prime factorisation stored as [(2, e_1), (3, e_2)...]

            # ----------------------------------------------------------------------
            # turn this into a list of factors with zeros included (maybe a better way?)
            canonical_g_k = g_k_factorisation
            prms = [p for p,e in g_k_factorisation]
            stop_prime = Primes().next(factor_base[-1])

            for premier in primes(2, stop=stop_prime):
                if premier not in prms:
                    canonical_g_k.append((premier,0))

            # sort this by the primes
            canonical_g_k = sorted(canonical_g_k, key=lambda tup: tup[0])
            # ----------------------------------------------------------------------

            # extract the exponents
            exponents = [e for p,e in canonical_g_k]
            exp_vector = vector(exponents + [k])

            # handle the case where it's empty list
            if len(relations) == 0:
                relations.append(exp_vector)

            # add it to end of relations
            relations_temp = matrix(relations).insert_row(len(relations), exp_vector)

            # check to see that this is lin ind with other relations
            if relations_temp.rank() == len(relations)+1:
                relations = list(relations_temp)

                # exit once there are at last r+1 relations
                if len(relations) >= len(factor_base):
                    break               
                    
    # obtain rref of relations matrix (last column is log_g(2), log_g(3), ...)
    relations = matrix(relations).rref()

    # extract last column and mod appropriately
    relations_last_col = vector([j % (q-1) for j in relations.column(-1)])

    # now we need to factor g^s*h mod q over the factor base
    s = 1
    gsh = g^s*h % q

    while not check_if_smooth(gsh, prd):    # loop until there is an s such that we have a factorization
        s += 1
        gsh = g^s*h % q
    
    
    gsh_factorisation = list(factor(gsh))

    # strip the primes
    prms2 = [p for p,e in gsh_factorisation]

    # strip the exponents
    exponents2 = [f for p,f in gsh_factorisation]

    # need zeros in the exponents...
    for index, p2 in enumerate(factor_base):
        if p2 not in prms2:
            exponents2.insert(index, 0)

    # cast as vector
    exponents2 = vector(exponents2)

    # compute the dot product of exponents and logs and discard s
    x = (exponents2.dot_product(relations_last_col) - s) % (q - 1)
    return x


We look at some examples

In [9]:
# toy examples
#1

q = 83
g = 2
h = 31
bound = 10

print(index_calc(g,q,h,bound))
# %timeit x = index_calc(g,q,h,bound)


38


In [10]:
# Check
g^38 % q

31

In [11]:
#2
q = 16785407
F = GF(q)
g = int(F.multiplicative_generator())
h = 99342
bound = 100
print(index_calc(g,q,h,bound))

9577170


In [13]:
# Check
g^9577170 % q

99342

In [15]:
#3
q = 1553012977
F = GF(q)
g = int(F.multiplicative_generator())
h = 993423443
bound = 100
print(index_calc(g,q,h,bound))

725427234
